<a href="https://colab.research.google.com/github/pujitha014/MACHINE_LEARNING/blob/main/diabetes_gaussian_prob.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df=pd.read_csv("diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
df.info()
x=df.drop("Outcome",axis=1).values
y=df["Outcome"].values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

x = scaler.fit_transform(x)
print(x)

[[ 0.63994726  0.84832379  0.14964075 ...  0.20401277  0.46849198
   1.4259954 ]
 [-0.84488505 -1.12339636 -0.16054575 ... -0.68442195 -0.36506078
  -0.19067191]
 [ 1.23388019  1.94372388 -0.26394125 ... -1.10325546  0.60439732
  -0.10558415]
 ...
 [ 0.3429808   0.00330087  0.14964075 ... -0.73518964 -0.68519336
  -0.27575966]
 [-0.84488505  0.1597866  -0.47073225 ... -0.24020459 -0.37110101
   1.17073215]
 [-0.84488505 -0.8730192   0.04624525 ... -0.20212881 -0.47378505
  -0.87137393]]


In [ ]:
from sklearn.model_selection import train_test_split
x=df.drop("Outcome",axis=1)
y=df["Outcome"]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [ ]:
def prior(y):
    # P(c) - Prior Class Probability
    k=pd.DataFrame(y)
    k=k.value_counts()
    print(k)
    for i in k.index:
      prior_prob[i[0]]=k[i[0]]/len(y)

prior_prob={}
prior(y_train)
print(prior_prob)


Outcome
0          401
1          213
dtype: int64
{0: 0.6530944625407166, 1: 0.3469055374592834}


In [ ]:
feature_values={}
#feature probability used for calculating likelihood
def feature_p(x,y):
    for i in np.unique(y):    #0,1
       example=x[y==i]    #all rows where y==0 once then y==1
       feature_values[i]={}   #create dictionary for 0,1 seperately
       for c in example.columns:
          # for each column find mean and variance - used for likelihood later
          feature_values[i][c]={
              'mean':example[c].mean(),
              'var':example[c].var()
          }

feature_p(x_train,y_train)
feature_values

{0: {'Pregnancies': {'mean': 3.2418952618453867, 'var': 9.158840399002493},
  'Glucose': {'mean': 110.214463840399, 'var': 678.3288902743142},
  'BloodPressure': {'mean': 68.30922693266832, 'var': 312.0441396508728},
  'SkinThickness': {'mean': 19.748129675810475, 'var': 212.6189027431421},
  'Insulin': {'mean': 72.25436408977556, 'var': 10233.220137157106},
  'BMI': {'mean': 30.256608478802992, 'var': 56.83436246882793},
  'DiabetesPedigreeFunction': {'mean': 0.4312543640897755,
   'var': 0.09430972013715712},
  'Age': {'mean': 30.556109725685786, 'var': 122.83746882793017}},
 1: {'Pregnancies': {'mean': 4.685446009389671, 'var': 13.09398529542032},
  'Glucose': {'mean': 140.88732394366198, 'var': 1070.1853574275844},
  'BloodPressure': {'mean': 71.49765258215963, 'var': 395.5341925768447},
  'SkinThickness': {'mean': 21.624413145539908, 'var': 285.3016653379396},
  'Insulin': {'mean': 98.72769953051643, 'var': 19298.010408362123},
  'BMI': {'mean': 35.234272300469485, 'var': 49.75858

In [ ]:
# general formula we multiply probabilities of each feature w.r.t yes/no but when log is applied we can directly add,
#why log applied ? because multiplication of small probabilities will result in very small values close to 0 values. applying log we can directly add
# denominator is same in bayesan formula for all yes/no predictions so ignored


classes=np.unique(y)
def predict(x):
   predictions = []
   for t,row in x.iterrows():   #iterate rows
       class_prob = {c: np.log(prior_prob[c]) for c in classes}  #initially prior probability
       for col in x.columns:
          #  print(x.loc[row,col])
           for f in classes:
              mean=feature_values[f][col]['mean']
              var=feature_values[f][col]['var']
              cell=row[col]
              likelihood = gaussian_likelihood(cell, mean, var)
              class_prob[f]+=np.log(likelihood)
              # print(class_prob[f])
       prediction=max(class_prob, key=class_prob.get)
       predictions.append(prediction)
   return np.array(predictions)


def gaussian_likelihood(x,mean,var):
    power=np.exp(-(x-mean)**2/(2*var))
    likelihood=power/np.sqrt(2*np.pi*var)
    return likelihood


p=predict(x_test)
# for i,j in zip(p,y):
#   print(i,j)

Naive Bayes 0.7662337662337663
